In [ ]:
import pyspark.sql.functions as F

## Read Data

In [ ]:
df_submission = spark.read.parquet("abfss://*******.dfs.core.windows.net/reddit/parquet/submissions") 

In [ ]:
df_comment = spark.read.parquet("abfss://*******.dfs.core.windows.net/reddit/parquet/comments") 

## Filter Data

In [ ]:
ru_subreddit_list = ['ukraine', 'ukraina', 'UkraineConflict',\
'RussiaUkraineWar2022', 'UkrainianConflict',\
'UkraineWarReports', 'UkraineInvasionVideos',\
'UkraineWarVideoReport']

war_subreddit_list = ['war', "CombatFootage", "CredibleDefense", "geopolitics"]
war_search_keywords = r'(^|\s)(ukraine|ukrainian|russia|russian|putin|zelenskyy)(\s|$)'

## Save Filtered Submission Data

In [ ]:
df_sub = df_submission.filter((F.col('subreddit').isin(ru_subreddit_list)) |
          ((F.col('subreddit').isin(war_subreddit_list)) & (F.lower(F.col('selftext')).rlike(war_search_keywords))))
df_sub = df_sub.withColumn("created_utc", F.from_unixtime("created_utc"))
df_sub = df_sub.withColumn("created_date", F.to_date("created_utc"))
df_sub = df_sub.filter(F.col("created_date") >= "2022-01-01")
df_sub.count()

Out[60]: 384972

In [ ]:
df_sub.write.mode('overwrite').format("parquet").save("/FileStore/tables/submissions_all")

## Save Filtered Comments Data

In [ ]:
df_com_all = df_comment.filter((F.col('subreddit').isin(ru_subreddit_list)) |
          ((F.col('subreddit').isin(war_subreddit_list)) & (F.lower(F.col('body')).rlike(war_search_keywords))))
df_com_all = df_com_all.withColumn("created_utc", F.from_unixtime("created_utc"))
df_com_all = df_com_all.withColumn("created_date", F.to_date("created_utc"))
df_com_all = df_com_all.filter(F.col("created_date") >= "2022-01-01")
df_com_all.count()

Out[62]: 9993790

In [ ]:
df_com_all.write.mode('overwrite').format("parquet").save("/FileStore/tables/comments_all")